<a href="https://colab.research.google.com/github/SakinduRansindu/Genarate_PDF_Images_From_CSV/blob/main/Genarate_Service_Letters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install & Import libraries**

---



---



## **Install libraries & Fonts**

In [ ]:
!pip install PyDrive2
!pip install reportlab
!pip install PyPDF2

In [ ]:
!pip install colorama

In [ ]:
!apt-get install -y fonts-dejavu-core

## **Import libraries**


In [ ]:
import pandas as pd

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import files
from google.colab.output import eval_js
import re

In [ ]:
from IPython.display import Image as IMG
from reportlab.lib import colors

In [ ]:
from PyPDF2 import PdfWriter, PdfReader
import time
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

In [ ]:
import numpy as np
import time
import subprocess
import os

In [ ]:
from colorama import Fore, Back, Style
from tqdm import tqdm

# **Function Definitions**

---



---



## **Utilities**

In [ ]:
def printMSG(msg):
  print(Fore.MAGENTA+Back.WHITE)
  print("\n________________________")
  print(msg)
  print("________________________\n")
  print(Style.RESET_ALL)

In [ ]:
def validateCSV_Cols(col):
  upper_case_cols = [ col.upper() for col in list(df.columns)]
  if col.upper() in upper_case_cols:
    return upper_case_cols.index(col.upper())
  else:
    return -1

In [ ]:
PUBLIC_URL = "about:blank"
def startFileServer():
    global PUBLIC_URL
    subprocess.Popen(["python", "-m", "http.server","8000"])
    PUBLIC_URL = eval_js("google.colab.kernel.proxyPort(8000)")

## **Google Services Setup**

In [ ]:
# Function to authenticate using the uploaded service account key
def authenticate_with_service_account(json_keyfile):
    scope = ['https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile,scope)
    gauth = GoogleAuth()
    gauth.credentials = credentials
    return GoogleDrive(gauth)

In [ ]:
def _downloadFile(file_id,downloadAs=None):
    # Fetch the file by ID
    file = drive.CreateFile({'id': file_id})
    # Read the content of the file
    if downloadAs !=None:
      filename = downloadAs + '.' + file['title'].split('.')[-1]
    else:
      filename = file['title']
    file.GetContentFile(filename)  # Downloads file with original title
    print(f"File '{filename}' has been downloaded.")
    return filename

In [ ]:
def _readFile(file_id):
  file = drive.CreateFile({'id': file_id})
  print(file.GetContentString())

In [ ]:
def _add_file_to_folder(file_path, folder_id):
    # Create a new file object
    new_file = drive.CreateFile({'parents': [{'id': folder_id}]})

    # Set the content of the file to be uploaded
    new_file.SetContentFile(file_path)

    # Upload the file to Google Drive
    new_file.Upload()

    # Return the file metadata (title and id)
    return {
        'title': new_file['title'],
        'id': new_file['id']
    }


In [ ]:
def _update_file(file_path, file_id):
    # Create a new file object
    new_file = drive.CreateFile({'id': file_id})

    # Set the content of the file to be uploaded
    new_file.SetContentFile(file_path)

    # Upload the file to Google Drive
    new_file.Upload()

    # Return the file metadata (title and id)
    return {
        'title': new_file['title'],
        'id': new_file['id']
    }


In [ ]:
def _extract_folder_id_from_url(folder_url_or_id):
    # Use a regular expression to extract the folder ID from the URL
    match = re.search(r'folders/([a-zA-Z0-9_-]+)', folder_url_or_id)
    match2 = re.search(r'([a-zA-Z0-9_-]+)', folder_url_or_id)

    if match:
        return match.group(1)  # Return the matched folder ID
    elif match2:
        return match2.group(1)
    else:
        raise ValueError("Invalid Google Drive folder URL format")

In [ ]:
def _extract_file_id(file_url_or_id):
    # If the input is already a file_id (only alphanumeric and hyphen/underscore characters)
    if re.fullmatch(r'[\w-]+', file_url_or_id):
        return file_url_or_id

    # Otherwise, try to extract file_id from a Google Drive URL
    pattern = r'(?:/d/|id=)([\w-]+)'
    match = re.search(pattern, file_url_or_id)

    if match:
        return match.group(1)
    else:
        raise ValueError("Invalid Google Drive file URL or file_id")

In [ ]:
def readFile(fileId_or_url):
    return _readFile(_extract_file_id(fileId_or_url))

def downloadFile(fileId_or_url,downloadAs=None):
    return _downloadFile(_extract_file_id(fileId_or_url),downloadAs)

def uploadFile(file_path,folder_url_or_id):
    return _add_file_to_folder(file_path,_extract_folder_id_from_url(folder_url_or_id))

def updateFile(file_path,fileId_or_url):
    file_id = _update_file(file_path,_extract_file_id(fileId_or_url))

## **Images And PDF File Genaration**

In [ ]:
def add_text_to_pdf(pdf_file, textObjs ,save_as="output.pdf"):
    original_pdf = PdfReader(pdf_file)
    writer = PdfWriter()
    # Create a temporary PDF to hold the text
    temp_pdf = "temp.pdf"
    c = canvas.Canvas(temp_pdf)

    for textObj in textObjs:
        label,text,x_coord,y_coord,font_size,font_color = textObj["label"],textObj["text"],textObj["x_coordinate"],textObj["y_coordinate"],textObj["font_size"],textObj["font_color"]
        font_type,text_alignment,page_numbers = textObj["font_type"],textObj["text_alignment"],textObj["pages"]
        for page_num in page_numbers:
            original_page = original_pdf.pages[page_num]
            original_page_size =tuple (original_pdf.pages[page_num]['/MediaBox'][2:])
            c.setPageSize(original_page_size)
            c.setFont(font_type, font_size)
            c.setFillColor(font_color)
            # c.drawString(x_coord, y_coord, text)
            if text_alignment == "C":
                c.drawCentredString(x_coord, y_coord, text)
            elif text_alignment == "R":
                c.drawRightString(x_coord, y_coord, text)
            elif text_alignment == "L":
                c.drawString(x_coord, y_coord, text)
    c.showPage()
    c.save()


    time.sleep(1)

    text_pdf = PdfReader(temp_pdf)

    # Overlay text on specified pages
    for page_num in page_numbers:
        original_page = original_pdf.pages[page_num]
        original_page.merge_page(text_pdf.pages[page_num])
        writer.add_page(original_page)

    # Save the modified PDF
    with open(save_as, "wb") as output_stream:
        writer.write(output_stream)


In [ ]:
def add_text_to_image(image_path,textObjs,saveAs="output_image.png"):
    try:
      img = Image.open(image_path)
      draw = ImageDraw.Draw(img)

      for textObj in textObjs:
          label,text,x,y= textObj["label"],textObj["text"],textObj["x_coordinate"],textObj["y_coordinate"]
          font_size,color,font_path,align = textObj["font_size"],textObj["font_color"],textObj["font_type"],textObj["text_alignment"]
          color = (color.red,color.green,color.blue)
          if font_path:
              print(font_path)
              font = ImageFont.truetype(font_path, font_size)
          else:
              printMSG("default font selected")
              font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", font_size)

          # Get text size using font.getbbox()
          bbox = font.getbbox(text)  # Returns (left, top, right, bottom)
          text_width = bbox[2] - bbox[0]
          text_height = bbox[3] - bbox[1]
          # Align text
          if align == "C":
              x -= text_width // 2  # Adjust for center alignment
          elif align == "R":
              x -= text_width  # Adjust for right alignment
          elif align =="L":
              pass
          else:
              print("warning: invalied text alignment. set to left align")

          draw.text((x, y), text, fill=color, font=font)

      img.save(saveAs)
      printMSG("Text added to the image successfully!")
    except Exception as e:
      print(f"Error adding text to image: {e}")


In [ ]:
def text_object_config(label,text,x_coordinate,y_coordinate,fontSize,color,align="L",font_type="Helvetica",appling_page_numbers=[0]):
  content_struct = {
    "label":label,
    "text":text,
    "x_coordinate":x_coordinate,
    "y_coordinate":y_coordinate,
    "font_size":fontSize,
    "font_color":color,
    "font_type":font_type,
    "text_alignment":align,
    "pages":appling_page_numbers
  }
  return content_struct

## **CSV Processing**

In [ ]:
def _find_longest_string_row_index(column_name):
  global df
  if column_name not in df.columns:
    raise IndexError(f"Error: Column '{column_name}' not found in DataFrame.")

  try:
    # Find the index of the row with the maximum string length in the specified column
    longest_string_index = df[column_name].str.len().idxmax()
    return longest_string_index
  except Exception as e:
    raise RuntimeError(f"Error finding longest string: {e}")


In [ ]:
def long_strings_onTextObjs(text_objs):
  return {_find_longest_string_row_index(c['label']) for c in text_objs}

In [ ]:
def minor_corrections():
  #validating columns
    global df
    if TEMPLATE_FILE_PDF!="":
        textObjsCount = len(text_items)
        text_items2 = text_items
    elif TEMPLATE_FILE_IMG!="":
        textObjsCount = len(textObjs)
        text_items2 = textObjs
    if textObjsCount<=0:
        raise ValueError("no column selected as text Objects")
    for i in range(textObjsCount):
        colname = text_items2[i]["label"]
        index = validateCSV_Cols(colname)
        if index < 0:
            raise ValueError(f"Invalid column name: {text_items2[i]['label']}")
        else:
            text_items2[i]["label"] = str(list(df.columns)[index])
    # add GenaratedFileURLs to csv
    if "GenaratedFileURLs" in list(df.columns):
        print("GenaratedFileURLs column already exists")
    else:
      df.loc[:,"GenaratedFileURLs"] = ""

In [ ]:
def addGenaratedFileURL(dictRecode,fileUrl):
    global df
    df.loc[dictRecode['_index'],"GenaratedFileURLs"]=fileUrl

In [ ]:
def getNextRecode():
    global df
    row = df[df["GenaratedFileURLs"]==""].iloc[0]
    dict_row = dict(row)
    dict_row["_index"]=row.name
    return dict_row

In [ ]:
def getRecodeByIndex(index):
    global df
    row = df.iloc[index]
    dict_row = dict(row)
    dict_row["_index"]=row.name
    return dict_row

In [ ]:
def w_case():
  minor_corrections()
  return df.iloc[list(long_strings_onTextObjs(text_items))]

## **Genarate A Single File**

In [ ]:
def genarateAndUpload_PDF(dictRecode,uploadAllowed=True):
    if TEMPLATE_FILE_PDF=="":
        raise ValueError("Template file url is empty")
    global df,text_items
    filename=str(time.asctime().replace(" ","_"))
    for txtitem in text_items:
        label = txtitem["label"]
        text = str(dictRecode[label])
        txtitem["text"] = text
    filename = str(dictRecode['_index'])+"_"+filename
    if uploadAllowed:
      add_text_to_pdf(pdf_Tfile,text_items,f"./pdfs/{filename}.pdf")
      details = uploadFile(f"./pdfs/{filename}.pdf",OUTPUT_FOLDER)
      FileUrl= "https://drive.google.com/file/d/"+details["id"]
      addGenaratedFileURL(dictRecode,FileUrl)
      return FileUrl
    else:
      global PUBLIC_URL
      add_text_to_pdf(pdf_Tfile,text_items,f"./temp/{filename}.pdf")
      return PUBLIC_URL+"temp/"+filename+".pdf"



In [ ]:
def genarateAndUpload_IMG(dictRecode,uploadAllowed=True):
    if TEMPLATE_FILE_IMG=="":
        raise ValueError("Template file url is empty")
    global df,textObjs
    filename=str(time.asctime().replace(" ","_"))
    ext = img_Tfile.split(".")[-1].lower()
    for txtitem in textObjs:
        label = txtitem["label"]
        text = str(dictRecode[label])
        txtitem["text"] = text
    filename = str(dictRecode['_index'])+"_"+filename+"."+ext
    print(filename)
    if uploadAllowed:
      add_text_to_image(img_Tfile,textObjs,f"{filename}")
      details = uploadFile(f"./imgs/{filename}",OUTPUT_FOLDER)
      FileUrl= "https://drive.google.com/file/d/"+details["id"]
      addGenaratedFileURL(dictRecode,FileUrl)
      return FileUrl
    else:
      global PUBLIC_URL
      add_text_to_image(img_Tfile,textObjs,f"./temp/{filename}")
      return PUBLIC_URL+f"/temp/{filename}"

## **Genarate All**

In [ ]:
def genarateAllFiles():
    minor_corrections()
    total_recodes = len(df[df["GenaratedFileURLs"]==""])
    completed_recodes = 0
    pbar = tqdm(total=total_recodes)
    while len(df[df["GenaratedFileURLs"]==""])>0:
        if TEMPLATE_FILE_PDF!="":
          furl = genarateAndUpload_PDF(getNextRecode())
          # print(f"complete {furl}")
        elif TEMPLATE_FILE_IMG!="":
          furl = genarateAndUpload_IMG(getNextRecode())
          # print(f"complete {furl}")
        df.to_csv(csv_file,index=False)
        updateFile(csv_file,CSV_FILE)
        pbar.update(1)

## **Testings**

In [ ]:
def wCaseTests():
    minor_corrections()
    if TEMPLATE_FILE_PDF!="":
      long_string_col_indexes = long_strings_onTextObjs(text_items)
    elif TEMPLATE_FILE_IMG!="":
      long_string_col_indexes = long_strings_onTextObjs(textObjs)
    print(text_items,textObjs)
    for index in long_string_col_indexes:
        if TEMPLATE_FILE_PDF!="":
          fid = genarateAndUpload_PDF(getRecodeByIndex(index),False)
          print(f"Sample PDF: {fid}")
        elif TEMPLATE_FILE_IMG!="":
          fid = genarateAndUpload_IMG(getRecodeByIndex(index),False)
          print(f"Sample Image: {fid}")

# **Configuring**

---



---



## **Set the CSV file & Template file urls or file IDs**


In [ ]:
!rm -r *

In [ ]:
printMSG("upload the json_keyfile")
json_keyfile = files.upload()
drive = None
if len(list(json_keyfile.keys())) > 0:
  json_keyfile_loc = '/content/'+list(json_keyfile.keys())[0]
  drive = authenticate_with_service_account(json_keyfile_loc)

In [ ]:
CSV_FILE = "https://drive.google.com/file/d/1he_GTj_2tFneth_bSQVI6KL2MUH12rwa/view?usp=sharing"
TEMPLATE_FILE_IMG = ""
TEMPLATE_FILE_PDF = "https://drive.google.com/file/d/1PvmJK8ttdPzvA0vF1BY3OfMlKyrvU1Ef/view?usp=drive_link"
OUTPUT_FOLDER = "https://drive.google.com/drive/folders/1lhAGXj4pKSjdrAA_asSeKnefxyxlugz2?usp=drive_link"

## **Download Templates And Data**

In [ ]:
if CSV_FILE!="":
    csv_file = downloadFile(CSV_FILE,"USERS")
else:
    raise ValueError("CSV file url is empty")
if TEMPLATE_FILE_IMG!="":
    img_Tfile = downloadFile(TEMPLATE_FILE_IMG,"T_IMG")
if TEMPLATE_FILE_PDF!="":
    pdf_Tfile = downloadFile(TEMPLATE_FILE_PDF,"T_PDF")
if TEMPLATE_FILE_IMG == "" and TEMPLATE_FILE_PDF == "":
    raise ValueError("At least one Template file url should be enterd")

## **Preview Data**

In [ ]:
df = None
def importCSV(ext):
  global df
  if ext.upper() in ["XLS","XLSX","XLSM","XLT"]:
    df = pd.read_excel("/content/USERS."+ext)
  elif ext.upper() == "CSV":
    df = pd.read_csv("/content/USERS."+ext)
  else:
    raise TypeError("Invalied file type. CSV file expected.")
importCSV(csv_file.split('.')[-1])

In [ ]:
df

## **Adjest Texts In The Sample**

In [ ]:
!mkdir "pdfs"
!mkdir "temp"
!mkdir "imgs"

In [ ]:
startFileServer()

In [ ]:
textObjs = []
img_sample = None
if TEMPLATE_FILE_IMG != "":
    ext = img_Tfile.split(".")[-1].lower()
    textObjs.append(text_object_config("first name","Text to show",1000,550,70,colors.Color(112,0,0),"C",None,[0]))
    textObjs.append(text_object_config("user id","20/10/2025",1000,600,20,colors.Color(0,1,0),"C",None,[0]))
    add_text_to_image(img_Tfile,textObjs,"temp/output_image"+"."+ext)
    print("sample image file preview: ",PUBLIC_URL+"temp/output_image"+"."+ext)
    img_sample = IMG(filename='temp/output_image.jpg', width='500')
img_sample

In [ ]:
text_items = []
if TEMPLATE_FILE_PDF != "":
    text_items.append(text_object_config("First Name","Sample Text 1",400,330,30,colors.Color(134,0,244),"C","Helvetica",[0]))
    text_items.append(text_object_config("email","20/12/2023",650,10,30,colors.Color(0,334,230),"C","Helvetica",[0]))

    add_text_to_pdf("T_PDF.pdf",text_items,"./temp/output_file.pdf")
    print("sample pdf file preview: ",PUBLIC_URL+"temp/output_file.pdf")

# **RUN**

---



---



In [ ]:
printMSG("importent: Check sample files are genarated properly".upper())
wCaseTests()

In [ ]:
genarateAllFiles()